# Climate Analysis with ClimRR

## Getting Started

::: {.callout-caution title="Useful links + References" collapse="true" width="100%"}

- Useful links + References:
    - [Geopandas: An Introduction](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-2/geopandas-an-introduction.html)
    - [An Introduction to Earth and Environmental Data Science](https://earth-env-data-science.github.io/intro.html)
        - [Final Projects](https://earth-env-data-science.github.io/projects.html) (good examples)
        - [Hands-On: Aggregating the Data](https://climateestimate.net/content/example-step3.html)
    - [Using Weather Data and Climate Model Output in Economic Analyses of Climate Change](https://www.journals.uchicago.edu/doi/10.1093/reep/ret016)
    - [On the use and misuse of climate change projections in international development](https://wires.onlinelibrary.wiley.com/doi/full/10.1002/wcc.579)

:::

::: {.callout-note title="Learning Goals" collapse="false" style="width:100%;"}

The goal of this project is to teach students to[^goals]:

- Use Unix commands to work with files and navigate directories
- Use JupyterHub + basic familiarity with how to use Jupyter notebooks on HPC systems
- Identify some of the common file types and data formats for geospatial data
    - (+ common python libraries for geospatial data analysis)
- Perform exploratory data analysis on geospatial data
    - (+ understand different operations for manipulating and interacting with this data)
    - tabular vs. gridded data
- Perform simple visualizations in Python to display different types of data (e.g. maps, line charts, interactive visualizations, etc)
    - using figures to illustrate a point or idea
    - know what types of plots to use for which situations / data types
- Understand control flow / basic structure of a Python script
    - using Python in Jupyter[^surprise]
    - `import`-ing libraries, etc

:::

[^surprise]: Getting Python setup correctly can be **surprisingly** difficult

[^goals]: Building on ideas from [Earth and Environmental Science](https://earth-env-data-science.github.io/intro.html)

### Reading and Writing Files

We use [`GeoPandas`](https://geopandas.org/en/stable/index.html), an open
source project to make working with geospatial data in python easier.

GeoPandas extends the datatypes used by [pandas](http://pandas.pydata.org/) to
allow spatial operations on geometric types.

Geometric operations are performed by [`shapely`](https://shapely.readthedocs.io/).

GeoPandas further depends on [`fiona`](https://fiona.readthedocs.io/) and
[`matplotlib`](http://matplotlib.org/) for plotting.

GeoPandas can read almost any vector-based spatial data format including ESRI
shapefile, GeoJSON files and more using the command


In [ ]:
import geopandas as gpd
import geodatasets
gdf = gpd.read_file(geodatasets.get_path("geoda.chicago_commpop"))
gdf.head(n=2)

which returns a GeoDataFrame object.

A `GeoDataFrame` is a tabular data structure that contains a `GeoSeries`.

The most important property of a `GeoDataFrame` is that it always has one
`GeoSeries` column that holds a special status.

This `GeoSeries` is referred to as the `GeoDataFrame`'s "geometry". When a
spatial method is applied to a `GeoDataFrame` (or a spatial attribute like area
is called), this commands will always act on the "geometry" column[^geometry].

[^geometry]: The “geometry” column – no matter its name – can be accessed
through the geometry attribute (`gdf.geometry`), and the name of the geometry
column can be found by typing `gdf.geometry.name`.

### Imports / setup

We've prepared some useful functions in
[`src/ClimRR/data.py`](https://github.com/saforem2/climate-analysis/blob/main/src/ClimRR/data.py)
to simplify the process of loading and working with our data.

<!-- ::: {.callout-warning title="Warning" collapse="false"} -->
<!---->
<!-- Make sure to follow the instructions from [Setup and -->
<!-- Installation](/qmd/GettingStarted/setup.qmd) for installing the `ClimRR` -->
<!-- package. -->
<!---->
<!-- ::: -->


In [ ]:
#| code-fold: true
#| code-summary: Imports
import matplotlib_inline
import matplotlib.pyplot as plt
import geopandas as gpd
import warnings

import matplotlib.pyplot as plt

from enrich.style import STYLES
from rich.theme import Theme
from rich.console import Console as Console
from ClimRR import get_logger, set_plot_style
from ClimRR.data import DATA_DIR

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

set_plot_style()
theme = Theme(STYLES)
log = get_logger()
console = Console(
    theme=Theme(STYLES),
    log_path=False,
    markup=True,
    width=512
)

# Data Organization

The [ClimRR Data](https://anl.box.com/s/hmkkgkrkzxxocfe9kpgrzk2gfc4gizp8) can
be downloaded as a `*.zip` file, and contains:  

- **Shapefile** (as an additional `*.zip` _inside_ the original)
- Individual `*.csv`'s for each of the climate variables
    - `{AnnualTemperatureMaximum,...,HeatingDegreeDays}.csv`

::: {.callout-tip title="`*.zip` contents:" collapse="true"}

```bash
📂 ClimRR Data Download/
┣━━ 📂 GridCells2Shapefile/
┃   ┣━━ 📄 GridCells2.cpg
┃   ┣━━ 📄 GridCells2.dbf
┃   ┣━━ 📄 GridCells2.prj
┃   ┣━━ 📄 GridCells2.sbn
┃   ┣━━ 📄 GridCells2.sbx
┃   ┣━━ 📄 GridCells2.shp
┃   ┣━━ 📄 GridCells2.shp.xml
┃   ┗━━ 📄 GridCells2.shx
┣━━ 📂 GridCellsShapefile/
┃   ┣━━ 📄 GridCells.cpg
┃   ┣━━ 📄 GridCells.dbf
┃   ┣━━ 📄 GridCells.prj
┃   ┣━━ 📄 GridCells.sbn
┃   ┣━━ 📄 GridCells.sbx
┃   ┣━━ 📄 GridCells.shp
┃   ┣━━ 📄 GridCells.shp.xml
┃   ┗━━ 📄 GridCells.shx
┣━━ 📄 AnnualTemperatureMaximum.csv
┣━━ 📄 AnnualTemperatureMinimum.csv
┣━━ 📄 ClimRR Metadata and Data Dictionary.pdf
┣━━ 📄 ConsecutiveDayswithNoPrecipitation.csv
┣━━ 📄 FireWeatherIndex_Wildfire.csv
┣━━ 📄 GridCells2Shapefile.zip
┣━━ 📄 GridCellsShapefile.zip
┣━━ 📄 HeatingDegreeDays.csv
┣━━ 📄 Precipitation_inches_AnnualTotal.csv
┣━━ 📄 README.txt
┣━━ 📄 SeasonalTemperatureMaximum.csv
┣━━ 📄 SeasonalTemperatureMinimum.csv
┗━━ 📄 WindSpeed.csv
```



:::

## Types of Data

::: {#fig-data-types}

![](../../assets/vector-data.png)

(Image credit: National Ecological Observatory Network (NEON))
:::

One of the most common file formats for vector data is the [ESRI
shapefile](https://en.wikipedia.org/wiki/Shapefile), which is what we will be
working with in be working with in this project.

### Metadata

Metadata is "data about the data"[^metadata] and is (by design) meant to give
additional information or provide context about a dataset.

Examples might include:

- When was this data created?
    - By who? For what? Where at? When? **Why**??
- How is this data licensed?
- Is there a reference for this data? (DOI ? URL ? etc.)
- What variables or fields are contained in this data?
    - What do they represent? Are there units?
- If the data is geospatial, what geographical or temporal area is included?
- Additional (often contextual) information about the data
    - e.g. "this data was created to inform a population about upcoming weather events" or similar


Metadata is often expected to be of a certain form, or to follow specific conventions / guidelines.

This is important to keep in mind and will allow others to understand your data
without needing an explanation (e.g. "what does this abbreviation mean?", "how
is this variable defined?", etc.)

Some common metadata conventions for GIS data include:

- [Climate and Forecast (CF) Conventions](https://cfconventions.org/)
- [Attribute Convention for Data Discovery](https://wiki.esipfed.org/Attribute_Convention_for_Data_Discovery_1-3)

[Schema.org](https://schema.org) is another useful reference and provides a
general framework for dealing with metadata.

## FAIR Data

[FAIR](https://www.force11.org/group/fairgroup/fairprinciples) stands for
"Findable, Accessible, Interoperable, Reusable", and provides a set
of guidelines for data sharing.

In the age of "big data", its important that we use (and promote!) tools that
facilitate the effective sharing of data.

Ideally, our data would be completely self-contained and provide, via
metadata, all of the information required to understand and work with it.

<!-- This is important to keep in mind as it allows others to understand and -->
<!-- interpret your data in a well-defined manner. -->
<!---->
<!-- There are oftentimes well-established conventions and formats that metadata is -->
<!-- expected to adhere to, so it is important to make -->



[^metadata]: [Metadata](https://earth-env-data-science.github.io/lectures/data.html#metadata)


## Load Shapefile and inspect

A `shapefile` is provided in the [ClimRR Data Download
(ANL)](https://anl.box.com/s/hmkkgkrkzxxocfe9kpgrzk2gfc4gizp8) and can be loaded using `geopandas.read_file(...)` which will return a `geopandas.GeoDataFrame`:


In [ ]:
shpfile = DATA_DIR.joinpath(
    "GridCellsShapefile/GridCells.shp"
)
shape = gpd.read_file(shpfile)

Each entry in this table defines a single **grid cell** (12km x 12 km) which
collectively tile the United States.

We can get a better understanding of whats going on by looking at the first few
entries:


In [ ]:
shape.head(n=2)

We see that each row has the following columns: `{OBJECTID, Crossmodel,
Shape_Leng, Shape_Area, geometry}`.

In particular, the `Crossmodel`[^crossmodel] column is a text ID that uniquely
identifies an individual cell.

[^crossmodel]: Truncated name for "Crossmodel_CellName".

To be explicit, let's look at the `WindSpeed.csv` file.


## Dealing with Geometry

Our shapefile contains a grid of _cells_ (12km x 12km) which tile the
continental US.

We can inspect a single cell:


In [ ]:
cell = shape[shape["Crossmodel"] == 'R146C497']
cell.head()

In [ ]:
ax = cell.boundary.plot()
ax.set_axis_off()
_ = ax.set_title('Grid from shapefile: 12 x 12 km')
plt.tight_layout()

In [ ]:
cell.explore()

## Load data from `*.csv` files

Each entry (row) in the `.csv` has a `Crossmodel` column (e.g. `R146C497`)
which corresponds to a row in our `shapefile` that uniquely determines its
location on the Earth.

We can associate with each of the `.csv`s the `geometry` used in our
`shapefile` to position our data on the globe.


In [ ]:
import pandas as pd
csvs = [i for i in DATA_DIR.rglob('*.csv')]
data = {}
for f in csvs:
    key = f.stem
    tmp = pd.read_csv(f.as_posix())
    gdf = shape.merge(tmp, on='Crossmodel')
    gdf['boundary'] = gdf.boundary
    gdf['centroid'] = gdf.centroid
    data[key] = gdf
    console.log(f"data['{key}'].shape={data[key].shape}")

## Look at the WindSpeed data

Lets inspect one of the entries in our `data[(...)]` dictionary, `WindSpeed`, for example:


In [ ]:
data["WindSpeed"].head()

We see that each entry has a `geometry` column, as well as columns for
`{hist,rcp45_midc, rcp45_endc, rcp85_midc, rcp85_endc, ...}` which contains the
numerical value of the `WindSpeed` in each cell under different scenarios at
different points in time.

<!-- We can see this explicitly by plotting this value for a single cell: -->

Let's look at the `WindSpeed` for our individual cell:


In [ ]:
cell_wind = data["WindSpeed"][data["WindSpeed"]["Crossmodel"] == 'R146C497']

In [ ]:
ax = cell_wind.plot(column='hist', legend=True)
ax.set_axis_off()
_ = ax.set_title("WindSpeed [Hist] for CELL: R146C497")

## Visualizing our Data

Lets inspect the first few entries from our Shapefile:

::: {#fig-increasing-cells style="text-align:left!important;"}


In [ ]:
#| code-fold: true
fig, ax = plt.subplots(
    figsize=(12, 3.5),
    nrows=1,
    ncols=3,
    sharey='row'
)
ax = ax.flatten()
pairs = {
    '1k': list(range(1000)),
    '5k': list(range(5000)),
    '20k': list(range(20000)),
}
for idx, (key, val) in enumerate(pairs.items()):
    ax[idx] = shape.loc[val, :].plot(ax=ax[idx])
    ax[idx].set_axis_off()
    _ = ax[idx].set_title(f"First {key} cells")
plt.tight_layout()

As we include more cells, we see the outline of the US beginning to take shape.
:::


[^venv]:
    Preferably, inside a virtual environment, e.g.
    ```bash
    $ mkdir -p ~/.venvs/ClimRR
    $ python3 -m venv ~/.venvs/ClimRR --system-site-packages
    $ source ~/.venvs/ClimRR/bin/activate
    $ python3 -m pip install git+https://github.com/saforem2/climate-analysis
    ```